In [1]:
#first step is installing all the requirements. 
#pip install urllib3
#pip install requests
#pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:

#import packages
from bs4 import BeautifulSoup
import requests
import requests.exceptions
from urllib.parse import urlsplit
from collections import deque
import re


In [3]:
new_urls = deque(['https://en.wikipedia.org/wiki/Email_address'])
processed_urls = set()

# a set of crawled emails
emails = set()

# process urls one by one until we exhaust the queue
while len(new_urls):
    # move next url from the queue to the set of processed urls
    url = new_urls.popleft()
    processed_urls.add(url)
    # get url's content
    print("Processing %s" % url)
    try:
        response = requests.get(url)
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
        # ignore pages with errors 
        continue
        
        # extract base url and path to resolve relative links
parts = urlsplit(url)
base_url = "{0.scheme}://{0.netloc}".format(parts)
path = url[:url.rfind('/')+1] if '/' in parts.path else url

Processing https://en.wikipedia.org/wiki/Email_address


In [4]:
# extract all email addresses and add them into the resulting set 
new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
emails.update(new_emails)
# create a beutiful soup for the html document
soup = BeautifulSoup(response.text)

# find and process all the anchors in the document
for anchor in soup.find_all("a"):
    # extract link url from the anchor
    link = anchor.attrs["href"] if "href" in anchor.attrs else ''
    # resolve relative links
    if link.startswith('/'):
        link = base_url + link
    elif not link.startswith('http'):
        link = path + link
    # add the new url to the queue if it was not enqueued nor processed yet
    if not link in new_urls and not link in processed_urls:
        new_urls.append(link)
        
        
        

In [5]:
for email in emails:
    print(email)

foobar@example.com
JSmith@example.com
user-@example.org
other.email-with-hyphen@example.com
x@example.com
1234567890123456789012345678901234567890123456789012345678901234+x@example.com
right@example.com
c@example.com
user@example.com
fully-qualified-domain@example.com
john.smith@example.org
disposable.style.email.with+symbol@example.com
example@s.example
l@example.com
simple@example.com
i_like_underscore@but_its_not_allowed_in_this_part.example.com
joeuser@example.com
myaddress+extension@pobox.com
very.common@example.com
user.name+tag+sorting@example.com
example.com@example.org
test@test.com
jsmith@example.com
example-indeed@strange-example.com
user.name@example.com
username@example.org
foo+bar@example.com
john.smith@example.com
joeuser+tag@example.com
John..Doe@example.com
allowed@example.com
20name+tag@domain.com
